In [61]:
import pickle
import pandas as pd
import numpy as np
from dota.persistence import load, save
from dota.data import json_to_dataframe

In [62]:
data = load('data/18.08-3.09.pkl')
# data = load('data/all_shit.pkl')

In [63]:
df = json_to_dataframe(data)

In [64]:
def split_picks_bans(df):
    df_picks_0 = df['picks_0'].apply(pd.Series)
    df_picks_0.columns = [f'pick_0_{x}' for x in range(1, 6)]

    df_picks_1 = df['picks_1'].apply(pd.Series)
    df_picks_1.columns = [f'pick_1_{x}' for x in range(1, 6)]


    df_bans_0 = df['bans_0'].apply(pd.Series)
    df_bans_0.columns = [f'ban_0_{x}' for x in range(1, 8)]

    df_bans_1 = df['bans_1'].apply(pd.Series)
    df_bans_1.columns = [f'ban_1_{x}' for x in range(1, 8)]
    
    return pd.concat([df_picks_0, df_picks_1, df_bans_0, df_bans_1], axis=1)
    

In [65]:
def get_all_heroes(df_heroes):
    all_heroes = set(df_heroes.stack().tolist())
    all_heroes.remove('')
    return sorted(all_heroes)

In [66]:
class FewHotEncoder:
    def __init__(self, words):
        self.list_of_words = sorted(set(words))
        self.empty_array = np.zeros(len(self.list_of_words))
    
    def encode(self, words):
        encoded = self.empty_array.copy()
        for word in words:
            encoded[self.list_of_words.index(word)] = 1
        return encoded
            

In [67]:
picks_bans_df = split_picks_bans(df)
all_heroes = get_all_heroes(picks_bans_df)

enc = FewHotEncoder(all_heroes)

In [68]:
encoded_picks_0 = df.picks_0.apply(lambda x: enc.encode(x).astype(int))
encoded_picks_1 = df.picks_1.apply(lambda x: enc.encode(x).astype(int))

encoded_picks = encoded_picks_0 - encoded_picks_1
encoded_picks = list(encoded_picks.apply(list))
encoded_picks = pd.DataFrame(encoded_picks)

In [69]:
df = df.drop(columns=['picks_0', 'picks_1', 'bans_0', 'bans_1'])

In [70]:
df = pd.concat([df, encoded_picks], axis=1)

In [71]:
X = df.drop(columns=['location',      'date',    'team_0',    'team_1', 'country_0',
       'country_1',    'winner', ])

In [72]:
Y = df.winner

In [73]:
X = pd.concat([X, X * (-1)])

In [74]:
Y = pd.concat([Y, 1-Y])

In [16]:
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [75]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np

cross_val_score(estimator=xgb.XGBClassifier(), X=X, y=Y,
                cv=5, scoring='accuracy')

/home/dpaluhin/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:35:02] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/dpaluhin/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:35:04] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/dpaluhin/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

/home/dpaluhin/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:35:10] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/dpaluhin/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:35:12] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


array([0.51630435, 0.5923913 , 0.57608696, 0.53551913, 0.58469945])

In [77]:
np.mean([0.51630435, 0.5923913 , 0.57608696, 0.53551913, 0.58469945])

0.561000238